## Stochastic ODE
$\frac{dy(t)}{dt} = -ky$ where decay rate coefficient $k$ is considered to be a random variable $k(\theta)$ with pdf $f(k)$ and mean value $\bar{k}$.

In [5]:
#Import necessary modules
import numpy as np
from scipy.sparse import diags, csr_matrix, kron
from scipy.linalg import solve_sylvester, schur, expm
from scipy.sparse.linalg import spsolve, cg
from scipy.integrate import quad
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
import time
import random
%config InlineBackend.figure_format = 'svg'

In [6]:
def plot_sol(X,Y,U,i):
    plt.figure(i)
    xline = np.reshape(X, -1)
    yline = np.reshape(Y, -1)
    zline = np.reshape(U, -1)
    plt.imshow(U, extent=[-1,1,-1,1], origin='lower')
    plt.colorbar()
    plt.axis(aspect='image')
    plt.xlabel('x')
    plt.ylabel('y')


def compute_err(U, U_exact):
    n = len(U)
    err_inf = 0
    err_sq = 0
    for i in range(0,n):
        for j in range(0,n):
            err_sq += np.absolute(U_exact[i][j] - U[i][j])**2
            if np.absolute(U_exact[i][j] - U[i][j]) > err_inf:
                err_inf = np.absolute(U_exact[i][j] - U[i][j])
        
    err_sq = (err_sq * h**2)**0.5
    return err_inf, err_sq


In [33]:
import numpy as np
from scipy.sparse import diags, kron
from scipy.sparse.linalg import spsolve, cg
from scipy.special import hermite, legendre
from scipy.integrate import quad
from sympy import *
from scipy import sparse

p = 3 #polynomial parameter
y = 1 #initial condition
h = 0.1 #time step

def sq_inner_prod(l):
    I = lambda k: hermite(l,k)*hermite(l,k)*(1/np.sqrt(2*np.pi))*np.exp(-k**2/2)
    return quad(I,-np.inf,np.inf)[0]

def e_inner_prod(i,j,l):
    I = lambda k: hermite(i,k)*hermite(j,k)*hermite(l,k)*(1/np.sqrt(2*np.pi))*np.exp(-k**2/2)
    return quad(I,-np.inf,np.inf)[0]

def k_top_inner_prod(i):
    I = lambda k: k*hermite(i,k)*(1/np.sqrt(2*np.pi))*(1/np.sqrt(2*np.pi))*np.exp(-k**2/2)
    return quad(I,-np.inf,np.inf)[0]

def f(y,t):
    rhs = np.zeros(p+1)
    for l in range(0,p):
        sum = 0
        for i in range(0,p):
            for j in range(0,p):
                sum += e_inner_prod(i,j,l)*(k_top_inner_prod(i)/sq_inner_prod(i))*y
        rhs[l] = -1/(sq_inner_prod(l)) * sum
    print(rhs)
    return rhs[0]
        
def rk2a( x0, t ):
    n = len( t )
    x = np.array( [ x0 ] * n )
    for i in range(0, n-1):
        h = t[i+1] - t[i]
        k1 = h * f( x[i], t[i] ) / 2.0
        x[i+1] = x[i] + h * f( x[i] + k1, t[i] + h / 2.0 )

    return x

t = np.linspace(0.1, 1, 10)
y = rk2a(1,t)
print(y)
            





[-0.79788456 -2.19418254 -0.2216346   0.        ]
[-0.76605357 -2.10664732 -0.21279266  0.        ]
[-0. -0. -0.  0.]
[-0. -0. -0.  0.]
[-0. -0. -0.  0.]
[-0. -0. -0.  0.]
[-0. -0. -0.  0.]
[-0. -0. -0.  0.]
[-0. -0. -0.  0.]
[-0. -0. -0.  0.]
[-0. -0. -0.  0.]
[-0. -0. -0.  0.]
[-0. -0. -0.  0.]
[-0. -0. -0.  0.]
[-0. -0. -0.  0.]
[-0. -0. -0.  0.]
[-0. -0. -0.  0.]
[-0. -0. -0.  0.]
[1 0 0 0 0 0 0 0 0 0]


In [32]:
import numpy as np
from scipy.sparse import diags, kron
from scipy.sparse.linalg import spsolve, cg
from scipy.special import hermite, legendre
from scipy.integrate import quad
from sympy import *
from scipy import sparse

p = 3 #polynomial parameter
y = 1 #initial condition
h = 0.1 #time step

def sq_inner_prod(l):
    I = lambda k: legendre(l,k)*legendre(l,k)*(1/2)
    return quad(I,-1,1)[0]

def e_inner_prod(i,j,l):
    I = lambda k: legendre(i,k)*legendre(j,k)*legendre(l,k)*(1/2)
    return quad(I,-1,1)[0]

def k_top_inner_prod(i):
    I = lambda k: k*legendre(i,k)*(1/np.sqrt(2*np.pi))*(1/2)
    return quad(I,-1,1)[0]

def f(y,t):
    rhs = np.zeros(p+1)
    for l in range(0,p):
        sum = 0
        for i in range(0,p):
            for j in range(0,p):
                sum += e_inner_prod(i,j,l)*(k_top_inner_prod(i)/sq_inner_prod(i))*y
        rhs[l] = -1/(sq_inner_prod(l)) * sum
    return rhs[0]
        
def rk2a( x0, t ):
    n = len( t )
    x = np.array( [ x0 ] * n )
    for i in range(0, n-1):
        h = t[i+1] - t[i]
        k1 = h * f( x[i], t[i] ) / 2.0
        x[i+1] = x[i] + h * f( x[i] + k1, t[i] + h / 2.0 )

    return x

t = np.linspace(0.1, 1, 10)
y = rk2a(1,t)
print(y)
            





[1 0 0 0 0 0 0 0 0 0]
